In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [48]:
from rake_nltk import Rake
rake_nltk_var = Rake()
text = """frog and dog is playing"""
rake_nltk_var.extract_keywords_from_text(text)
keyword_extracted = rake_nltk_var.get_ranked_phrases()
print(keyword_extracted)

['playing', 'frog', 'dog']


In [4]:
# nltk.download("tag")

In [49]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))


tokenized = word_tokenize(text)

tagged = nltk.pos_tag(tokenized)
print(tagged)

[('frog', 'NN'), ('and', 'CC'), ('dog', 'NN'), ('is', 'VBZ'), ('playing', 'VBG')]


In [51]:

#for finding the sentiment of the sentence
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
def retSetiment(text1):
    analyzer = SentimentIntensityAnalyzer()

    user_input = text1
    
    sentiment_scores = analyzer.polarity_scores(user_input)

    if sentiment_scores['compound'] >= 0.05:
        sentiment = "positive"
    elif sentiment_scores['compound'] <= -0.05:
        sentiment = "negative"
    else:
        sentiment = "neutral"

    print(f"Sentiment: {sentiment}")
    return sentiment

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [53]:
import json
import re
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

def appendAdj(extNoun):
    extNoun=extNoun.lower()
    animals_adj = open("./data.json")
    ani_json = json.load(animals_adj)
    name_adj_dict = {}
    ani_json = ani_json['animals']
    sentSentiment = retSetiment(text) #negative
    currAdj = []
    for i in ani_json:
        if(i['name'] == extNoun):
            for j in  range(len(i['adjectives'])):
                if(i['adjectives'][j]["type"] == sentSentiment):
                    currAdj.append(i['adjectives'][j]["value"])
                    
    name_adj_dict[extNoun] = currAdj
   
    return name_adj_dict[extNoun]
    
    
    
allAdj = {}
allText = []

k=0

for i,j in (tagged):
    if(j=='NN'):
        allAdj[i] = appendAdj(i)
# for i in allAdj:
#     print(i," ",allAdj[i])
allText.append([])
allText[0]=[text]
length=1
for k in allAdj:
    allText.append([])
    if(allAdj[k] == []):
        continue
    for v1 in allAdj[k]:
        for m in allText[length-1]:
            v1 += " " + k
            allText[length].append(re.sub(k,v1,m))
    length+=1
for i in allText[length-1]:
    print(i+"\n")

Sentiment: positive
Sentiment: positive
colorful frog and cute dog is playing

vibrant frog and cute dog dog is playing

striking frog and cute dog dog dog is playing

tropical frog and cute dog dog dog dog is playing

colorful frog and active dog is playing

vibrant frog and active dog dog is playing

striking frog and active dog dog dog is playing

tropical frog and active dog dog dog dog is playing

colorful frog and happy dog is playing

vibrant frog and happy dog dog is playing

striking frog and happy dog dog dog is playing

tropical frog and happy dog dog dog dog is playing

colorful frog and obedient dog is playing

vibrant frog and obedient dog dog is playing

striking frog and obedient dog dog dog is playing

tropical frog and obedient dog dog dog dog is playing



In [56]:
import json
import joblib
from HabitatPred import preprocess_text

fdata = open('./data.json')
fdata = json.load(fdata)
fdata = fdata['animals']

def checkAndReturnHabitat(extNoun):
    for i in fdata:
        if(i['name'] == extNoun):
            # print(i['habitats'])
            return i['habitats']
    return []

def isHabitat(extNoun):
    model = joblib.load('habitatModel.pkl')
    vectorizer = joblib.load('vectorModel.pkl')

    user_input = extNoun

    user_input_preprocessed = preprocess_text(user_input)

    user_input_vec = vectorizer.transform([user_input_preprocessed])

    prediction = model.predict(user_input_vec)

    if prediction[0] == 1:
        return True
    return False
    
#habitat inclusion
currAllText = allText[length-1]
newAllText = []
setHabitats = []

flag = True
for i,j in (tagged):
    if ((j == 'NN') and (isHabitat(j) == False)):
        flag = False
    else:
        flag = True
        break
    
#if there is no habitat found in the sentence
if flag == False:
    for i,j in (tagged):
        if ((j == 'NN') and (isHabitat(j) == False)): #dog
            setHabitats = checkAndReturnHabitat(i)
            if(setHabitats == []):
                continue
            for l in currAllText:
                for k in setHabitats:
                    temp = l
                    temp += ", in the " + k
                    newAllText.append(temp)
    for i in newAllText:
        print(i,end = "\n")


ModuleNotFoundError: No module named 'pandas'

In [9]:
# nltk.download('stopwords')

In [10]:
# print(text)
def cosineTop10(allText1):
    maxDict1 = {}
    top10Sentences = []
    for i in allText1:
        col1=[i]
        col2=[text]
        vectors1 = model.encode(col1, convert_to_tensor=True)
        vectors2 = model.encode(col2, convert_to_tensor=True)
        cosine_scores = util.cos_sim(vectors1, vectors2)
        for j,(sent1,sent2) in enumerate(zip(col1,col2)):
            if cosine_scores[j][j]>=0.5:
                maxDict1[i] = cosine_scores[j][j]
    l1 = 10
    ind1 = 0
    for k, v in sorted(maxDict1.items(), key=lambda item: item[1],reverse=True):
        if(ind1 >= l1 or ind1 >= len(maxDict1)):
            break
        ind1+=1
        top10Sentences.append(k)
    return top10Sentences 
topSentences = cosineTop10(allText[length-1])
    

In [11]:
for i in topSentences:
    print(i,end = "\n")

tiny frog is crying, in a forest


Dependecy identification and Tree Generation

tiny: amod -> frog
frog: nsubj -> crying
is: aux -> crying
crying: ROOT -> crying
,: punct -> crying
in: prep -> crying
a: det -> forest
forest: pobj -> in


In [13]:
import spacy

nlp = spacy.load("en_core_web_sm")
sentence = "cat in the house and drinking milk which is stolen by other cats"
doc = nlp(sentence)
print(doc.ext)

AttributeError: 'spacy.tokens.doc.Doc' object has no attribute 'ext'

In [ ]:
#Algorithm implementation
import json
class Node:
    def __init__(self,val,adj):
        self.val = val #string
        self.adj = adj #[]
    def getVal(self):
        return self.val

class GoT:
    def __init__(self,arr): # arr contains nsubj, dobj
        
        self.children = []
    def createGraph(self):
        for i in self.data_json:
            n = Node(i["name"])
            self.children.append(Node("adjectives"))
            self.children.append(Node("habitats"))
                    
                

got = GoT(["Tiger","lion"])
got.createGraph()

Frog
{'name': 'Frog', 'adjectives': ['colorful', 'poisonous', 'tiny', 'vibrant', 'striking', 'tropical'], 'habitats': ['rainforests', 'jungles', 'tropical forests']}
{'name': 'Eagle', 'adjectives': ['majestic', 'imposing', 'predatory', 'large', 'powerful', 'elegant'], 'habitats': ['tropical rainforests', 'dense jungles', 'remote forested regions']}
{'name': 'Jaguar', 'adjectives': ['powerful', 'ferocious', 'spotted', 'solitary', 'elusive', 'carnivorous'], 'habitats': ['rainforests', 'swamps', 'grasslands']}
{'name': 'Toucan', 'adjectives': ['colorful', 'tropical', 'vibrant', 'beak-shaped', 'noisy', 'fruit-eating'], 'habitats': ['rainforests', 'tropical forests', 'jungles']}
{'name': 'Sloth', 'adjectives': ['slow-moving', 'tree-dwelling', 'peaceful', 'herbivorous', 'folivorous', 'sleepy'], 'habitats': ['tropical rainforests', 'canopy', 'forested regions']}
{'name': 'Orangutan', 'adjectives': ['intelligent', 'solitary', 'arboreal', 'ginger-haired', 'strong', 'endangered'], 'habitats': ['

In [ ]:
class Node:
    def __init__(self, name, node_type):
        self.name = name
        self.node_type = node_type
        self.children = []
    def add_child(self, child):
        self.children.append(child)
    def __str__(self) -> str:
        return f"{self.name} - ({self.node_type})"
   
class Graph:
    def __init__(self):
        self.nodes = []

    def add_node(self, node):
        self.nodes.append(node)

    def add_edge(self, parent, child):
        parent.add_child(child)

    def get_children(self, node):
        return node.children
    def findNode(self,animalName):
        for i in self.nodes:
            if(i.node_type == 'animal' and i.name == animalName):
                temp =  self.get_children(i)
                return temp
        return []
# Create a graph
graph = Graph()
data = open('./data.json')

data_json = json.load(data)

data_json = data_json['animals']

root = Node("animal", "root")
graph.add_node(root)
for i in range(len(data_json)):
    n1 = Node(data_json[i]['name'],"animal")
    graph.add_node(n1)
    for adj in data_json[i]["adjectives"]:
        adjective_node = Node(adj, "adjective")
        graph.add_node(adjective_node)
        graph.add_edge(n1, adjective_node)

    for habitat in data_json[i]["habitats"]:
        habitat_node = Node(habitat, "habitat")
        graph.add_node(habitat_node)
        graph.add_edge(n1, habitat_node)

testAnimal = "Giraffe"
cdren = graph.findNode(testAnimal)
if(cdren != []):
    for i in cdren:
        print(i)
else:
    print("Not Found")

tall - (adjective)
graceful - (adjective)
herbivorous - (adjective)
spotted - (adjective)
gentle - (adjective)
long-necked - (adjective)
African savannah - (habitat)
woodlands - (habitat)
grasslands - (habitat)
